In [5]:
import pandas as pd

In [20]:
EURUSD = pd.read_csv('D:\MixBollinger\EURUSD_220710~/MixBollinger_EURUSD.csv')

In [21]:
EURUSD

,SystemDate,Date,DST,Week,OpenTime,CloseTime,OpenPrice,ClosePrice,Type,Profit,MaxProfit,Rate,MaxMinTime,LosCut,Spread,OpenTime(2),CenterTime
0,2010.01.04 10:36,2010.01.04,N,1,10:36,16:19,1.43799,1.44043,0,24.4,69.9,34.9,280,47.213813,1.4,2010/1/4 10:36,2010/1/5 9:04
1,2010.01.05 05:25,2010.01.05,N,2,5:25,9:02,1.44281,1.44404,0,12.3,54.9,22.4,156,14.173674,1.6,2010/1/5 5:25,2010/1/5 9:04
2,2010.01.05 12:05,2010.01.05,N,2,12:05,12:39,1.44097,1.44244,1,-14.7,8.8,-167.0,19,14.771980,1.3,2010/1/5 12:05,2010/1/5 12:39
3,2010.01.05 13:42,2010.01.05,N,2,13:42,14:38,1.44398,1.44258,0,-14.0,14.2,-98.6,5,14.891786,1.4,2010/1/5 13:42,2010/1/5 14:38
4,2010.01.05 15:01,2010.01.05,N,2,15:01,15:14,1.44108,1.44260,1,-15.2,3.6,-422.2,3,15.116385,1.3,2010/1/5 15:01,2010/1/5 15:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14689,2022.07.07 17:00,2022.07.07,Y,4,17:00,21:23,1.01806,1.01644,1,16.2,35.6,45.5,203,10.638527,1.6,2022/7/7 17:00,2022/7/8 3:32
14690,2022.07.08 04:16,2022.07.08,Y,5,4:16,5:01,1.01920,1.01738,0,-18.2,0.0,0.0,0,17.079843,1.8,2022/7/8 4:16,2022/7/8 5:01
14691,2022.07.08 06:03,2022.07.08,Y,5,6:03,9:23,1.01573,1.01651,1,-7.8,16.9,-46.2,65,15.375405,1.6,2022/7/8 6:03,2022/7/8 9:23
14692,2022.07.08 10:09,2022.07.08,Y,5,10:09,13:09,1.01529,1.01296,1,23.3,80.1,29.1,35,10.777972,1.6,2022/7/8 10:09,2022/7/8 14:47


In [22]:
EURUSD.dtypes

SystemDate      object
Date            object
DST             object
Week             int64
OpenTime        object
CloseTime       object
OpenPrice      float64
ClosePrice     float64
Type             int64
Profit         float64
MaxProfit      float64
Rate           float64
MaxMinTime       int64
LosCut         float64
Spread         float64
OpenTime(2)     object
CenterTime      object
dtype: object

In [93]:
#日付ごとに集計するための日付リストを作成

# 使用するモジュールのインポート
from datetime import datetime as dt
from datetime import timedelta

# 日付条件の設定
strdt = dt.strptime("2010-01-01", '%Y-%m-%d')  # 開始日
enddt = dt.strptime("2022-07-8", '%Y-%m-%d')  # 終了日

# 日付差の日数を算出（リストに最終日も含めたいので、＋１しています）
days_num = (enddt - strdt).days + 1  # （参考）括弧の部分はtimedelta型のオブジェクトになります

# シンプルにforとappendを使用した場合
datelist = []
for i in range(days_num):
    dt = strdt + timedelta(days=i)
    if(dt.strftime("%a") == 'Sat' or dt.strftime("%a") == 'Sun'):
        continue
    
    dateinfo = []
    
    dateinfo.append(dt)
    dateinfo.append(dt.strftime("%Y.%m.%d"))
    dateinfo.append(dt.strftime("%a"))
    
    datelist.append(dateinfo)
    
    
datelist_df = pd.DataFrame(datelist, columns = ['date(datetime)', 'date(Str)', 'week'])
datelist_df

,date(datetime),date(Str),week
0,2010-01-01,2010.01.01,Fri
1,2010-01-04,2010.01.04,Mon
2,2010-01-05,2010.01.05,Tue
3,2010-01-06,2010.01.06,Wed
4,2010-01-07,2010.01.07,Thu
...,...,...,...
3261,2022-07-04,2022.07.04,Mon
3262,2022-07-05,2022.07.05,Tue
3263,2022-07-06,2022.07.06,Wed
3264,2022-07-07,2022.07.07,Thu


In [90]:
datelist_df.dtypes

date(datetime)    datetime64[ns]
date(Str)                 object
week                      object
dtype: object